In [50]:
import pandas as pd

filename = 'cpp/output.csv'

with open(filename, 'r') as file:
    lines = file.readlines()

# Split the lines based on the empty line separator
table_lines = []
current_table = []

for line in lines:
    if line.strip():
        current_table.append(line)
    elif current_table:
        table_lines.append(current_table)
        current_table = []

tables = []

for lines in table_lines:
    table = [line.strip().split(',') for line in lines]
    df = pd.DataFrame(table[1:], columns=table[0])
    columns_to_drop = ['bytes_per_second', 'items_per_second', 'label', 'error_occurred', 'error_message']
    df = df.drop(columns=columns_to_drop)
    tables.append(df)
tables.append(df)
print('C++ Benchmarks')
table_names = ["Implementation 1: build eertree and use it for a string of size n",
              "Implemenation 2: build eertree and use it for a string of size n",
               "Implemenation 1: number of rich strings of size n",
                "Implementation 2: number of rich strings of size n" ]
# Access each DataFrame separately
for idx, df in enumerate(tables):
    print(f"--- {table_name} ---")
    print(table_names[idx])

    table_style = df.style.set_properties(**{'text-align': 'center'})

    # Define a color function to apply to the cells
    def color_cell(value):
        if value == "s" or "bench" in value:
            return 'color: green'
        if value.isdigit():
            return 'color: yellow'
        return 'color: white'

    table_style = table_style.applymap(color_cell)

    display(table_style)
    print()

C++ Benchmarks
--- C++ Benchmarks ---
Implementation 1: build eertree and use it for a string of size n


,name,iterations,real_time,cpu_time,time_unit
0,"""bench/10000/repeats:20_mean""",20,0.00171328,0.00171298,s
1,"""bench/10000/repeats:20_median""",20,0.00167459,0.00167458,s
2,"""bench/10000/repeats:20_stddev""",20,0.000134664,0.000134022,s
3,"""bench/10000/repeats:20_cv""",20,0.00393002,0.00391195,s
4,"""bench/20000/repeats:20_mean""",20,0.00884285,0.00884218,s
5,"""bench/20000/repeats:20_median""",20,0.00875711,0.00875675,s
6,"""bench/20000/repeats:20_stddev""",20,0.000210781,0.00021095,s
7,"""bench/20000/repeats:20_cv""",20,0.00119181,0.00119286,s
8,"""bench/40000/repeats:20_mean""",20,0.032094,0.0320923,s
9,"""bench/40000/repeats:20_median""",20,0.03211,0.0321091,s



--- C++ Benchmarks ---
Implemenation 2: build eertree and use it for a string of size n


,name,iterations,real_time,cpu_time,time_unit
0,"""bench/10000/repeats:100_mean""",100,0.00180429,0.001804,s
1,"""bench/10000/repeats:100_median""",100,0.00180885,0.0018088,s
2,"""bench/10000/repeats:100_stddev""",100,0.000135756,0.000135533,s
3,"""bench/10000/repeats:100_cv""",100,0.000752405,0.000751293,s
4,"""bench/20000/repeats:100_mean""",100,0.00310612,0.00310602,s
5,"""bench/20000/repeats:100_median""",100,0.00310638,0.00310628,s
6,"""bench/20000/repeats:100_stddev""",100,4.60458e-05,4.60377e-05,s
7,"""bench/20000/repeats:100_cv""",100,0.000148242,0.000148221,s
8,"""bench/40000/repeats:100_mean""",100,0.00591131,0.00591093,s
9,"""bench/40000/repeats:100_median""",100,0.0058496,0.00584955,s



--- C++ Benchmarks ---
Implemenation 1: number of rich strings of size n


,name,iterations,real_time,cpu_time,time_unit
0,"""bench/2/repeats:5_mean""",5,0.000573875,0.000573864,s
1,"""bench/2/repeats:5_median""",5,0.000576384,0.000576375,s
2,"""bench/2/repeats:5_stddev""",5,1.40383e-05,1.40392e-05,s
3,"""bench/2/repeats:5_cv""",5,0.00489247,0.00489286,s
4,"""bench/4/repeats:5_mean""",5,0.00171814,0.00171772,s
5,"""bench/4/repeats:5_median""",5,0.00172325,0.00172319,s
6,"""bench/4/repeats:5_stddev""",5,4.51792e-05,4.59329e-05,s
7,"""bench/4/repeats:5_cv""",5,0.00525909,0.00534811,s
8,"""bench/6/repeats:5_mean""",5,0.00639421,0.00639348,s
9,"""bench/6/repeats:5_median""",5,0.00631983,0.0063191,s



--- C++ Benchmarks ---
Implementation 2: number of rich strings of size n


,name,iterations,real_time,cpu_time,time_unit
0,"""bench/2/repeats:5_mean""",5,0.000573875,0.000573864,s
1,"""bench/2/repeats:5_median""",5,0.000576384,0.000576375,s
2,"""bench/2/repeats:5_stddev""",5,1.40383e-05,1.40392e-05,s
3,"""bench/2/repeats:5_cv""",5,0.00489247,0.00489286,s
4,"""bench/4/repeats:5_mean""",5,0.00171814,0.00171772,s
5,"""bench/4/repeats:5_median""",5,0.00172325,0.00172319,s
6,"""bench/4/repeats:5_stddev""",5,4.51792e-05,4.59329e-05,s
7,"""bench/4/repeats:5_cv""",5,0.00525909,0.00534811,s
8,"""bench/6/repeats:5_mean""",5,0.00639421,0.00639348,s
9,"""bench/6/repeats:5_median""",5,0.00631983,0.0063191,s


In [58]:
from tabulate import tabulate
from colorama import Fore, Style

def parse_benchmark_output(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    benchmark_data = []
    current_benchmark = None
    table = []

    for line in lines:
        line = line.strip()

        if line.startswith("benchmarking"):
            if current_benchmark is not None:
                benchmark_data.append([current_benchmark] + table)
                table = []
            current_benchmark = line.split()[1:]
        elif line.startswith(("time", "mean", "std dev", "variance")):
            parts = line.split()
            field = parts[0]
            value = " ".join(parts[1:])
            table.append(value)

    if current_benchmark is not None:
        benchmark_data.append([current_benchmark] + table)

    return benchmark_data

def create_pretty_table(benchmark_data):
    headers = [Fore.YELLOW + "Benchmark Name", Fore.YELLOW + "Time", Fore.YELLOW + "Mean",
               Fore.YELLOW + "Std Dev", Fore.YELLOW + "Variance" + Style.RESET_ALL]

    tables = []
    prev_benchmark_name = None
    current_table = []
    for row in benchmark_data:
        benchmark_name = row[0]
        if prev_benchmark_name is None or benchmark_name[0] != prev_benchmark_name[0]:
            if current_table:
                tables.append(tabulate(current_table, headers, tablefmt="pipe"))
                current_table = []
        current_table.append([list(benchmark_name)] + [Fore.LIGHTCYAN_EX + value + Style.RESET_ALL for value in row[1:]])
        prev_benchmark_name = benchmark_name

    if current_table:
        tables.append(tabulate(current_table, headers, tablefmt="pipe"))

    return tables

file_path = "hs/output.txt"
benchmark_data = parse_benchmark_output(file_path)
tables = create_pretty_table(benchmark_data)
for table in tables:
    print(table)


| Benchmark Name                  | Time                            | Mean                            | Std Dev                             | Variance                                          |
|:--------------------------------|:--------------------------------|:--------------------------------|:------------------------------------|:--------------------------------------------------|
| ['eertree', '@2/len', '10000']  | 158.9 ms (129.1 ms .. 197.5 ms) | 133.0 ms (122.0 ms .. 143.4 ms) | dev 18.62 ms (11.54 ms .. 29.82 ms) | introduced by outliers: 48% (moderately inflated) |
| ['eertree', '@2/len', '20000']  | 279.6 ms (204.1 ms .. 327.0 ms) | 268.7 ms (251.1 ms .. 280.1 ms) | dev 19.60 ms (9.993 ms .. 29.99 ms) | introduced by outliers: 18% (moderately inflated) |
| ['eertree', '@2/len', '40000']  | 583.5 ms (294.3 ms .. 753.9 ms) | 500.1 ms (434.5 ms .. 536.8 ms) | dev 64.33 ms (2.110 ms .. 84.68 ms) | introduced by outliers: 24% (moderately inflated) |
| ['eertree', '@2/len', '80000